# **Machine Transliteration**

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64 # Batch size for Training
epochs = 100 # Number of epochs to train for
latent_dim = 256 #Latent dimentionality of the encoding space
num_sample = 15000 # Number of samples to train
#here change the path accordingly
data_path = 'crowd_transliterations.hi-en.txt'

In [ ]:
#preprocessing or cleaning of data

def preprocess_sentence_english(w):
  for ch in w:
    if((ch >= 'a' and ch <= 'z') or (ch >= 'A' and ch <= 'Z')):
      {}
    else:
      return 'false'
  return 'true'

In [ ]:
# vectorize the data 

input_texts = [] # english list
target_texts = [] # Hindi list
input_characters = set() #distinct input characters
target_characters = set() #distinct output characters
with open(data_path, 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')
for line in lines[:len(lines)-1]:
  input_text,target_text = line.split('\t');
  res = preprocess_sentence_english(input_text);
  if(res == 'false'):
    continue

  target_text = '\t' + target_text + '\n'
  input_texts.append(input_text);
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char);
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char);

In [ ]:
input_characters = sorted(list(input_characters));
target_characters = sorted(list(target_characters));

#required for the creation of 3D arrays for LSTM
num_encoder_tokens = len(input_characters)
print(num_encoder_tokens);
num_decoder_tokens = len(target_characters)
print(num_decoder_tokens);
max_encoder_seq_length = max([len(word) for word in input_texts])
print(max_encoder_seq_length);
max_decoder_seq_length = max([len(word) for word in target_texts])
print(max_decoder_seq_length);

In [ ]:
# here we are providing the index to the input tokens and target tokens
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [ ]:
# initialize arrays with all zeroes to store one hot representation
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype = 'float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype = 'float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype = 'float32')

In [ ]:
# create one hot representation
    for i, (ip_text, tar_text) in enumerate(zip(input_texts, target_texts)):
      for t, char in enumerate(ip_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
      
      for t, char in enumerate(tar_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
          decoder_target_data[i, t-1, target_token_index[char]] = 1.

In [ ]:
# encoder LSTM's
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state = True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c] #because we did not required intermediate encoder outputs

In [ ]:
# decoder LSTM's
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
# model formation
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
r = model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size = batch_size, 
          epochs=epochs, 
          validation_split=0.2)

In [ ]:
import matplotlib.pyplot as plt

# plot some data
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

# accuracies
plt.plot(r.history['accuracy'], label='acc')
plt.plot(r.history['val_accuracy'], label='val_acc')
plt.legend()
plt.show()

In [ ]:
#creation of encoder model and decoder model 

encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,)) #decoders hidden state
decoder_state_input_c = Input(shape=(latent_dim,)) #decoders cell state
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

#reverse indexing in order to predicr output
reverse_input_char_index = dict((i, char) for char,i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char,i in target_token_index.items())

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, 0, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
# Testing
for seq_index in range(100):
  input_seq = encoder_input_data[seq_index: seq_index+1]
  decoded_word = decode_sequence(input_seq)
  print('Input word : ', input_texts[seq_index])
  print('Output word : ', decoded_word);

In [ ]:
# user input
user_In = input("enter input : ")
inp = np.zeros((1,max_encoder_seq_length, num_encoder_tokens), dtype = 'float32')
for t, char in enumerate(user_In):
  inp[0,t, input_token_index[char]] = 1.
decoded_word = decode_sequence(inp)
print('Input word : ', user_In)
print('Output word : ', decoded_word)